### LSTM

##### Gate

![1677396390017](image/LSTM/1677396390017.png)

1. Cell State
   1. Gates

      1. Forgets gate Layer [$\sigma$]

      $$
      _t = \sigma(W_f \cdot [h_{t-1},x_t] + b_f) \cdots (1)
      $$

      - $\sigma$ - Layer로 이루어져 있다, 출력 값은 0(버려라) or 1(기억해라)로 이루어져 있다.
      - 특정 cell 이 주어를 포함하고 있을 수 있다.
        - ex) John (남성 표현) likes Pizza, So he (남성 대명사) ... 에서 'John'을 통해 'He'를 예측 가능하게 된다.

      2. **Input Gate Layer**
      3. [$\sigma$] 어떤 값을 Update 할 지 결정한다.

         $$
         i_t = \sigma (W_i \cdot [h_{t-1},x_t] + b_i) \cdots(2)
         $$
      4. $[\tanh]$ Cell State에 더해 질 수 있는 새로운 후보 값을 생성한다.

         $$
         \tilde{C}_t = \tanh(W_c \cdot [h_{t-1},x_t] + b_C) \cdots(3)
         $$
      5. 위 세 개의 식을 지나서 새로운 Cell을 만들어 냅니다.

      $$
      C_t = f_t * C_{t-1} + i_t * \tilde{C}_t
      $$

      4. 새롭게 만든 $C_t$에 새로운 필터링을 적용하여 $h_t$로 내보내고, 다음 $C_t$에 새롭게 적용 시킵니다.

      $$
      o_t = (W_o[h_{t-1},x_t]+b_o)
      $$

      $$
      h_t = o_t * \tanh(C_t)
      $$
2. 이 외에도 여러 Custimized LSTM이 존재하고,  좋은 모델을 찾아서 적용하는게 좋다.

#### 데이터 분할

1. Normal
   1. 학습용 데이터 ($S_N$)
      1. 학습 시킨 데이터를 추후 Reconstruction 할 때 쓰임
   2. 생성용 데이터 (Threshold : $V_{N_1}$)
   3. 검증용 데이터 ($V_{N_2}$)
   4. 실험용 데이터 ($T_N$)
2. Abnormal
   1. 검증용 데이터 ($V_A$)
   2. 실험용 데이터 ($T_A$)

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import torch as t 
import torch.nn as nn

from tqdm.notebook import trange, tqdm
from torch.utils.data import DataLoader, Dataset
from celluloid import Camera
from typing import List


In [3]:
## Data Load
df = pd.read_csv('./kaggle/sensor.csv', index_col=0)

## Check the Data
df.head()

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,...,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status
0,2018-04-01 00:00:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
1,2018-04-01 00:01:00,2.465394,47.09201,53.2118,46.310760,634.3750,76.45975,13.41146,16.13136,15.56713,...,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL
2,2018-04-01 00:02:00,2.444734,47.35243,53.2118,46.397570,638.8889,73.54598,13.32465,16.03733,15.61777,...,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL
3,2018-04-01 00:03:00,2.460474,47.09201,53.1684,46.397568,628.1250,76.98898,13.31742,16.24711,15.69734,...,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL
4,2018-04-01 00:04:00,2.445718,47.13541,53.2118,46.397568,636.4583,76.58897,13.35359,16.21094,15.69734,...,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL


1. Label을 3가지로 나누어서 본다.
   1. Normal
   2. Broken
   3. Recovering
2. 센서데이터를 시각화 하여 고장구간을 표시한다.